# Team

<a target="_blank" href="https://colab.research.google.com/drive/1iuP5kw92Qa48QMUJW-K50r-pSouXYgD-?usp=sharing"> <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Helder Mateus dos Reis Matos

Choose a problem in the [Integer Programming Exercise](ilp-exec-2.ipynb) and choose a strategy presented in the [Heuristics Lecture](../lectures/03_01_Heuristics.ipynb), you must write a python code that implements a solution to this problem following the chosen strategy.

Also solve the problem using ```pyomo``` and the GLPK or COIN-OR solvers.

Please send your Colab file (.ipynb) in SIGAA. The assignment can be done in pairs.

In [1]:
# !pip install -q pyomo
# !apt install -q -y glpk-utils

In [2]:
from typing import List, Optional, Tuple, Dict
import math

import pyomo.environ as pyo

# Problem 1

FFB fresh fruit business mixes apples, peaches, and nectarines to make three different types of baskets for local market. Each basket contains approximately 5 kg fruits. The content of three types of baskets is specified as follows:

|Basket Type|Apple|Peach|Nectarine|
|--|--|--|--|
|1|At least 30%|At most 20%|-|
|2|-|At most 40%|At least 20%|
|3|At least 20%|-|At most 30%|

FFB purchases apple at a cost of $1.00/kg, peach $1.50/kg, and nectarine $1.80/kg, and sells type-1 basket at $2.25/kg, type-2 basket $3.00/kg, and type-3 basket $2.60/kg. The daily supply of fruit is limited to 60 kg of apples, 70 kg of peaches, and 50 kg of nectarines.  FFB is able to sell all the fruit baskets they prepare for a given day.

Formulate an ILP model to determine how the fruit be mixed in order to maximize the profit. Solve it.

## 1.1. Solver solution with GLPK

O objetivo é maximizar o lucro, definido como a diferença entre a receita total e o custo total:

$$\max Z = 2.25(5y_{1}) + 3.00(5y_{2}) + 2.60(5y_{3}) - (1.00\sum a_{j} + 1.50\sum p_{j} + 1.80\sum n_{j})$$

Onde:

- $y_j$: número de cestas do tipo $j$ (1, 2 ou 3)
- $a_j, p_j, n_j$: quilogramas de apples, peaches, e nectarines na cesta do tipo $j$

Cada cesta deve pesar 5kg e seguir as seguintes restrições:

- Tipo 1: Apple $\geq$ 30%, Peach $\leq$ 20%
- Tipo 2: Peach $\leq$ 40%, Nectarine $\geq$ 20%
- Tipo 3: Apple $\geq$ 20%, Nectarine $\leq$ 30%

Para as restrições de suprimento:

$$a_1 + a_2 + a_3 \leq 60$$

$$p_1 + p_2 + p_3 \leq 70$$

$$n_1 + n_2 + n_3 \leq 50$$

Um `ConcreteModel()` é criado para armazenar as informações do problema. A lista `types` representa os três tipos de cestas e é representado por um conjunto do Pyomo.

In [3]:
model_basket = pyo.ConcreteModel()

types = [1, 2, 3]
model_basket.types = pyo.Set(initialize=types)

As variáveis de decisão correspondem diretamente às escolhas de produção e composição. A variável inteira `y` indica quantas cestas de cada tipo serão utilizadas, enquanto que as variáveis inteiras `a`, `p` e `n` representam os quilogramas de appes, peaches e nectarienes usados em cada tipo de cesta.

In [4]:
model_basket.y = pyo.Var(model_basket.types, domain=pyo.NonNegativeIntegers)
model_basket.a = pyo.Var(model_basket.types, domain=pyo.NonNegativeIntegers)
model_basket.p = pyo.Var(model_basket.types, domain=pyo.NonNegativeIntegers)
model_basket.n = pyo.Var(model_basket.types, domain=pyo.NonNegativeIntegers)

A variável `basket_weight` estabelece que cada cesta pesa 5 quilogramas.

O dicionário `supply` especifica a disponibilidade diária de cada tipo de fruta em quilogramas.

O dicionário `cost` representa o custo para comprar um quilograma de cada fruta.

O dicionario `price` representa o preço de venda de cada tipo de cesta.

In [5]:
basket_weight = 5.0
supply = {"a": 60.0, "p": 70.0, "n": 50.0}
cost = {"a": 1.00, "p": 1.50, "n": 1.80}
price_per_kg = {1: 2.25, 2: 3.00, 3: 2.60}

Já para as restrições, precisamos garantir uma consistência entre as quantidades de frutas usadas e o número de cestas produzidas. A função `weight_rule()` retorna um booleano que responde se, para um tipo de cesta `j`, a soma dos quilogramas de apples, peaches e nectarines (`m.a[j] + m.p[j] + m.n[j]`) é igual ao peso total de cestas deste tipo (`basket_mass * m.y[j]`).

Essa restrição é aplicada a todos os tipos de cesta, garantindo que não importa como o otimizador combina as frutas, cada cesta sempre respeita o limite de 5 kg.

In [6]:
def weight_rule(m, j):
    return m.a[j] + m.p[j] + m.n[j] == basket_weight * m.y[j]

model_basket.weight = pyo.Constraint(model_basket.types, rule=weight_rule)

Para cada tipo de fruta, são aplicados limites superiores e inferiores sobre a percentagem necessária de cada fruta em cada tipo de cesta.

In [7]:
model_basket.types1_a_min = pyo.Constraint(expr=model_basket.a[1] >= 0.30 * basket_weight * model_basket.y[1])
model_basket.types1_p_max = pyo.Constraint(expr=model_basket.p[1] <= 0.20 * basket_weight * model_basket.y[1])

model_basket.types2_p_max = pyo.Constraint(expr=model_basket.p[2] <= 0.40 * basket_weight * model_basket.y[2])
model_basket.types2_n_min = pyo.Constraint(expr=model_basket.n[2] >= 0.20 * basket_weight * model_basket.y[2])

model_basket.types3_a_min = pyo.Constraint(expr=model_basket.a[3] >= 0.20 * basket_weight * model_basket.y[3])
model_basket.types3_n_max = pyo.Constraint(expr=model_basket.n[3] <= 0.30 * basket_weight * model_basket.y[3])

Além disso, deve ser reforçado que o total de quilogramas de frutas usadas não exceda as disponibilidades diárias. O modelo soma os pesos de apples, peaches e nectarines usados em todos os tipos e compara cada total ao seu respectivo limite no dicionário `supply`. Isso previne que o modelo aloque mais frutas que o disponibilizado no estoque.

In [8]:
model_basket.apple_sup  = pyo.Constraint(expr=sum(model_basket.a[j] for j in model_basket.types) <= supply["a"])
model_basket.peach_sup  = pyo.Constraint(expr=sum(model_basket.p[j] for j in model_basket.types) <= supply["p"])
model_basket.nec_sup    = pyo.Constraint(expr=sum(model_basket.n[j] for j in model_basket.types) <= supply["n"])

Por fim, a função objetivo realiza a maximização do lucro diário. A variável `revenue` representa o valor total das cestas vendidas, enquanto que os custos totais das compras de apples, peaches e nectarines são agregados em `fruit_cost`. Assim, a função objetivo é declarada como `revenue - fruit_cost`.

In [9]:
revenue = sum(price_per_kg[j] * basket_weight * model_basket.y[j] for j in model_basket.types)

cost_apples = sum(model_basket.a[j] for j in model_basket.types) * cost["a"]
cost_peaches = sum(model_basket.p[j] for j in model_basket.types) * cost["p"]
cost_nectarines = sum(model_basket.n[j] for j in model_basket.types) * cost["n"]

fruit_cost = cost_apples + cost_peaches + cost_nectarines

model_basket.obj = pyo.Objective(expr=revenue - fruit_cost, sense=pyo.maximize)

O lucro máximo obtido é de \$285, onde a melhor combinação concentra a produção totalmente em cestas do tipo 2, com `y[2] = 36` cestas (180 kg). O modelo usa todo o estoque de frutas: `a[2] = 60 kg`, `p[2] = 70 kg`, `n[2] = 50 kg`.

As restrições das cestas de tipo 2 são satisfeitas da seguinte forma:

- peaches possuem $70/180 \approx 38.9\%$ (abaixo do máximo de 40%)
- nectarines possuem $50/180 \approx 27.8\%$ (acima do mínimo de 20%)
- apples possuem $\approx 33.3\%$.

A cesta de tipo 2 tem o maior preço de venda por quilograma e contém todas as frutas sem violar os limites de percentagem. Alocar qualquer cesta do tipo 1 ou 3 só iria reduzir o lucro.

In [10]:
solver = pyo.SolverFactory("glpk")
solver.solve(model_basket, tee=False)

model_basket.display()

Model unknown

  Variables:
    y : Size=3, Index=types
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
          2 :     0 :  36.0 :  None : False : False : NonNegativeIntegers
          3 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
    a : Size=3, Index=types
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
          2 :     0 :  60.0 :  None : False : False : NonNegativeIntegers
          3 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
    p : Size=3, Index=types
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
          2 :     0 :  70.0 :  None : False : False : NonNegativeIntegers
          3 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
    n : Size=3, Index=types
        Key

## 1.2. Heuristic solution by divide and conquer

In [11]:
basket_weight = 5
cost_fruits = {"apple": 1.00, "peach": 1.50, "nectarine": 1.80}
price_baskets = {1: 2.25, 2: 3.00, 3: 2.60}

initial_supply = {"apple": 60, "peach": 70, "nectarine": 50}

In [12]:
from heuristic import heuristic_divide_and_conquer

In [13]:
sol, rem, profit = heuristic_divide_and_conquer(initial_supply, cost_fruits, price_baskets, basket_weight)
print("Heuristic solution (divide & conquer):\n")
counts = {1: 0, 2: 0, 3: 0}
usage = {"apple": 0, "peach": 0, "nectarine": 0}
for b in sol:
    counts[b.type_id] += 1
    usage["apple"] += b.apple
    usage["peach"] += b.peach
    usage["nectarine"] += b.nectarine
for i in (1, 2, 3):
    print(f"Type {i}: {counts[i]} baskets")
print("\nTotal usage (kg):", usage)
print("Remaining supply (kg):", rem)
print(f"\nTotal baskets: {len(sol)}  |  Total profit: ${profit:.2f}")
print("\nSample baskets:")
for b in sol[:10]:
    print(b, f" profit=${b.profit():.2f}")

Heuristic solution (divide & conquer):

Type 1: 0 baskets
Type 2: 0 baskets
Type 3: 0 baskets

Total usage (kg): {'apple': 0, 'peach': 0, 'nectarine': 0}
Remaining supply (kg): {'apple': 60, 'peach': 70, 'nectarine': 50}

Total baskets: 0  |  Total profit: $0.00

Sample baskets:
